Web Scraping

// Install libary 

In [1]:
// pip install beautifulsoup4 requests pandas

     |████████████████████████████████| 112kB 3.3MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Import library
import bs4
import pandas as pd
import requests

In [3]:
'''Let’s connect to the IMDB top 100 movies webpage, extract the HTML 
behind it and convert it to a BeautifulSoup object'''

url = 'https://www.imdb.com/search/title/?count=100&groups=top_1000&sort=user_rating'

In [4]:
def get_page_contents(url):
    page = requests.get(url, headers={"Accept-Language": "en-US"})
    return bs4.BeautifulSoup(page.text, "html.parser")

In [5]:
soup = get_page_contents(url)

In [7]:
# Since we’ve already located the HTML tag containing each movie card, we can get a list of all distinct movies and their corresponding HTML by

movies = soup.findAll('h3', class_='lister-item-header')


In [8]:
titles = [movie.find('a').text for movie in movies]

In [15]:
release = [movie.find('span', class_='lister-item-year text-muted unbold').text for movie in movies]

In [26]:
def numeric_value(movie, tag, class_=None, order=None):
    if order:
        if len(movie.findAll(tag, class_)) > 1:
            to_extract = movie.findAll(tag, class_)[order]['data-value']
        else:
            to_extract = None
    else:
        to_extract = movie.find(tag, class_)['data-value']

    return to_extract


In [27]:
def text_value(movie, tag, class_=None):
    if movie.find(tag, class_):
        return movie.find(tag, class_).text
    else:
        return

In [28]:
def nested_text_value(movie, tag_1, class_1, tag_2, class_2, order=None):
    if not order:
        return movie.find(tag_1, class_1).find(tag_2, class_2).text
    else:
        return [val.text for val in movie.find(tag_1, class_1).findAll(tag_2, class_2)[order]]


In [29]:
def extract_attribute(soup, tag_1, class_1='', tag_2='', class_2='', text_attribute=True, order=None, nested=False):
    movies = soup.findAll('div', class_='lister-item-content')
    data_list = []
    for movie in movies:
        if text_attribute:
            if nested:
                data_list.append(nested_text_value(movie, tag_1, class_1, tag_2, class_2, order))
            else:
                data_list.append(text_value(movie, tag_1, class_1))
        else:
            data_list.append(numeric_value(movie, tag_1, class_1, order))

    return data_list


In [30]:
titles = extract_attribute(soup, 'a')
release = extract_attribute(soup, 'span', 'lister-item-year text-muted unbold')
audience_rating = extract_attribute(soup, 'span', 'certificate')
runtime = extract_attribute(soup, 'span', 'runtime')
genre = extract_attribute(soup, 'span', 'genre')
imdb_rating = extract_attribute(soup, 'div', 'inline-block ratings-imdb-rating', False)
votes = extract_attribute(soup, 'span' , {'name' : 'nv'}, False, 0)
earnings = extract_attribute(soup, 'span' , {'name' : 'nv'}, False, 1)
directors = extract_attribute(soup, 'p', '', 'a', '', True, 0, True)
actors = extract_attribute(soup, 'p', '', 'a', '', True, slice(1, 5, None), True)


In [31]:
df_dict = {'Title': titles, 'Relase': release, 'Audience Rating': audience_rating,
           'Runtime': runtime, 'Genre': genre, 'IMDB Rating': imdb_rating,
           'Votes': votes, 'Box Office Earnings': earnings, 'Director': directors,
           'Actors': actors}

In [32]:
df = pd.DataFrame(df_dict)

In [34]:
df.head()

,Title,Relase,Audience Rating,Runtime,Genre,IMDB Rating,Votes,Box Office Earnings,Director,Actors
0,The Shawshank Redemption,(1994),R,142 min,\nDrama,\n\n9.3\n,"2,179,377","2,179,377",Frank Darabont,"[Tim Robbins, Morgan Freeman, Bob Gunton, Will..."
1,The Godfather,(1972),R,175 min,"\nCrime, Drama",\n\n9.2\n,"1,501,329","1,501,329",Francis Ford Coppola,"[Marlon Brando, Al Pacino, James Caan, Diane K..."
2,The Dark Knight,(2008),PG-13,152 min,"\nAction, Crime, Drama",\n\n9.0\n,"2,162,870","2,162,870",Christopher Nolan,"[Christian Bale, Heath Ledger, Aaron Eckhart, ..."
3,The Godfather: Part II,(1974),R,202 min,"\nCrime, Drama",\n\n9.0\n,"1,051,225","1,051,225",Francis Ford Coppola,"[Al Pacino, Robert De Niro, Robert Duvall, Dia..."
4,The Lord of the Rings: The Return of the King,(2003),PG-13,201 min,"\nAdventure, Drama, Fantasy",\n\n8.9\n,"1,550,568","1,550,568",Peter Jackson,"[Elijah Wood, Viggo Mortensen, Ian McKellen, O..."
